# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Blogpost:
Discover the path to mastering deep learning with DeepLearning.AI, a leading online platform founded by AI expert Andrew Ng. Whether you're a beginner or an experienced professional, DeepLearning.AI offers top-tier courses to elevate your skills in artificial intelligence and deep learning. From neural networks to natural language processing, the platform provides comprehensive learning opportunities to guide you through the complexities of AI technology. Join a global community of learners and enhance your expertise with practical knowledge and hands-on experience. Embark on your journey to unlock the full potential of deep learning with DeepLearning.AI.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Blogpost:
Discover the path to mastering deep learning with DeepLearning.AI, a leading online platform founded by AI expert Andrew Ng. Whether you're a beginner or an experienced professional, DeepLearning.AI offers top-tier courses to elevate your skills in artificial intelligence and deep learning. From neural networks to natural language processing, the platform provides comprehensive learning opportunities to guide you through the complexities of AI technology. Join a global community of learners and enhance your expertise with practical knowledge and hands-on experience. Embark on your journey to unlock the full potential of deep learning with DeepLearning.A

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [15]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [16]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Blogpost:
Discover the path to mastering deep learning with DeepLearning.AI, a leading online platform founded by AI expert Andrew Ng. Whether you're a beginner or an experienced professional, DeepLearning.AI offers top-tier courses to elevate your skills in artificial intelligence and deep learning. From neural networks to natural language processing, the platform provides comprehensive learning opportunities to guide you through the complexities of AI technology. Join a global community of learners and enhance your expertise with practical knowledge and hands-on experience. Embark on your journey to unlock the full potential of deep learning with DeepLearning.A


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

After aggregating the feedback from the SEO reviewer, Legal reviewer, and Ethics reviewer, here are some final suggestions on the writing regarding DeepLearning.AI:

1. **SEO Aspect**:
   - The content should include target keywords like "Deep Learning courses" or "AI online learning" in the title and throughout the content for better search engine visibility. 
   - Incorporating internal links to relevant pages within the website can improve site structure and user engagement.
   - Utilize meta tags, such as meta descriptions, with relevant keywords to enhance click-through rates from search engine results pages.

2. **Legal Considerations**:
   - Ensure that the use of the term "DeepLearning.AI" complies with intellectual property rights associated with the brand, such as trademarks or copyrights.
   - Check for any false advertising claims or misleading statements about the 

## Get the summary

In [17]:
print(res.summary)

Title: "Master Deep Learning: Your Guide to DeepLearning.AI Courses"

Blogpost:
Dive into the world of AI education with DeepLearning.AI, a premier online platform curated by renowned AI expert Andrew Ng. Explore cutting-edge Deep Learning courses tailored for all skill levels, from novice to expert. Elevate your expertise in neural networks and natural language processing through hands-on learning. Join a global community of learners and unlock the full potential of AI technology. Embrace a transformative learning journey with DeepLearning.AI today!

---
Adjusted the title and content to better suit SEO guidelines, legal considerations, and ethical guidelines. Maintained a concise and engaging tone while addressing feedback from reviewers.
